In [31]:
import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

from sklearn.model_selection import train_test_split

In [32]:
# Preparing the Dataset

X_train_full, X_test, y_train_full, y_test = train_test_split( housing.data,
         housing.target.reshape(-1, 1), random_state=42)

X_train, X_valid, y_train, y_valid =  train_test_split( X_train_full, 
        y_train_full, random_state=42)

In [33]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [34]:
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [35]:
def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < 1
        squared_loss = tf.square(error) / 2
        linear_loss  = tf.abs(error) - 0.5
        return tf.where(is_small_error, squared_loss, linear_loss)

In [36]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="selu", 
       kernel_initializer="lecun_normal", 
       input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])

In [40]:
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])

In [41]:
model.fit(X_train_scaled, y_train, epochs=1, validation_data=(X_valid_scaled, y_valid))

363/363 [==============================] - 2s 2ms/step - loss: 0.5782 - mae: 0.9332 - val_loss: 0.3311 - val_mae: 0.6253
